In [98]:
import pandas as pd
import os
from IPython.display import display
from pandasql import sqldf
import numpy as np
dataset_path = 'dataset'
pysqldf = lambda q: sqldf(q, globals())


silver_df = pd.read_parquet(os.path.join(dataset_path, 'silver_df.parquet'))


In [99]:
# Like for 3_descriptive_analysis
# Extract the number of different athletes for each NOC per Year for Summer Olympics

count_distinct_athletes = f" SELECT NOC, Year, COUNT(DISTINCT ID) as distinct_athletes_no" \
                f" FROM silver_df" \
                f" WHERE Season='Summer' " \
                 f" GROUP BY NOC, Year"
gold_distinct_athletes_df = pysqldf(count_distinct_athletes)



In [100]:
# Validate the previous query by looking at the number of distinct athletes for team USA
# In the last 3 editions (2016, 2012, 2008)

# Results should be:
# - 2016: 554 (263 men and 291 women) in 33 sports
# - 2012: 530 (262 men and 268 women) in 31 sports
# - 2008: 588 (306 men and 282 women) in 32 sports

# display(gold_distinct_athletes_df.loc[gold_distinct_athletes_df.NOC == 'USA'])
# Query result: 555 (+1), 530, 588.
# The team USA website reports 558 participants for Rio 2016


In [101]:
# Extract the number of competitions entries for each NOC per Year for Summer Olympics

count_events_partecipations = f" SELECT NOC, Year, COUNT(1) as events_partecipations_no" \
                f" FROM silver_df" \
                f" WHERE Season='Summer' " \
                 f" GROUP BY NOC, Year"
gold_events_partecipations_df = pysqldf(count_events_partecipations)

# display(gold_events_partecipations_df.loc[gold_events_partecipations_df.NOC == 'USA'])


In [102]:
# Extract number of medals by type for each NOC


subquery_count_winning = f" SELECT NOC, Sport, Event, Year, Season, Games, Team, Medal" \
                f" FROM silver_df" \
                f" WHERE Medal IS NOT NULL AND Season='Summer' " \
                 f" GROUP BY NOC, Sport, Event, Year, Season, Games, Team, Medal" \
                f" ORDER BY NOC"
medalists_df = pysqldf(subquery_count_winning)


In [103]:

# NOC, year, and different medals
noc_medals_df = pd.pivot_table(medalists_df,
                               index=['NOC', 'Year'],
                               columns='Medal',
                               aggfunc='count',
                               values='Event',
                               fill_value=0,
                               margins=True,
                               margins_name='TotalPartecipants')

noc_medals_df = noc_medals_df.drop(('TotalPartecipants',''))
noc_medals_df.columns.name = ''
noc_medals_df.reset_index(inplace=True)
display(noc_medals_df)


,NOC,Year,Bronze,Gold,NoMedal,Silver,TotalPartecipants
0,AFG,1936,0,0,4,0,4
1,AFG,1948,0,0,2,0,2
2,AFG,1956,0,0,1,0,1
3,AFG,1960,0,0,13,0,13
4,AFG,1964,0,0,8,0,8
...,...,...,...,...,...,...,...
2805,ZIM,2000,0,0,19,0,19
2806,ZIM,2004,1,1,8,1,11
2807,ZIM,2008,0,1,11,3,15
2808,ZIM,2012,0,0,8,0,8


In [104]:
# gold_partecipants_df contains NOC, Year, events_partecipations_no, distinct_athletes_no, Bronze, Silver, Gold, Total
gold_partecipants_df = gold_events_partecipations_df.merge(gold_distinct_athletes_df, on=['NOC', 'Year'], how='left')
gold_partecipants_df = gold_partecipants_df.merge(noc_medals_df, on=['NOC', 'Year'], how='left')
gold_partecipants_df = gold_partecipants_df.fillna(value=0)
gold_partecipants_df.Gold = gold_partecipants_df.Gold.astype(int)
gold_partecipants_df.Silver = gold_partecipants_df.Silver.astype(int)
gold_partecipants_df.Bronze = gold_partecipants_df.Bronze.astype(int)
gold_partecipants_df.TotalPartecipants = gold_partecipants_df.TotalPartecipants.astype(int)
gold_partecipants_df['TotalMedals'] = gold_partecipants_df.Gold + gold_partecipants_df.Silver + gold_partecipants_df.Bronze
gold_partecipants_df = gold_partecipants_df.loc[gold_partecipants_df.Year > 1960]
display(gold_partecipants_df)

,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals
4,AFG,1964,8,8,0,0,8,0,8,0
5,AFG,1968,5,5,0,0,5,0,5,0
6,AFG,1972,8,8,0,0,8,0,8,0
7,AFG,1980,11,11,0,0,11,0,11,0
8,AFG,1988,5,5,0,0,5,0,5,0
...,...,...,...,...,...,...,...,...,...,...
2805,ZIM,2000,26,16,0,0,19,0,19,0
2806,ZIM,2004,14,12,1,1,8,1,11,3
2807,ZIM,2008,16,13,0,1,11,3,15,4
2808,ZIM,2012,9,7,0,0,8,0,8,0


In [105]:
# Read NOC table
noc_df = pd.read_parquet(os.path.join(dataset_path, 'iso_countries.parquet'))
gold_partecipants_df = gold_partecipants_df.merge(noc_df, on=['NOC'], how='left')
gold_partecipants_df.fillna('Not Found')
display(gold_partecipants_df)

,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names
0,AFG,1964,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan
1,AFG,1968,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan
2,AFG,1972,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan
3,AFG,1980,11,11,0,0,11,0,11,0,Afghanistan,AFG,Afghanistan
4,AFG,1988,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,ZIM,2000,26,16,0,0,19,0,19,0,Zimbabwe,ZWE,Zimbabwe
2176,ZIM,2004,14,12,1,1,8,1,11,3,Zimbabwe,ZWE,Zimbabwe
2177,ZIM,2008,16,13,0,1,11,3,15,4,Zimbabwe,ZWE,Zimbabwe
2178,ZIM,2012,9,7,0,0,8,0,8,0,Zimbabwe,ZWE,Zimbabwe


In [106]:
# Regions not found
display(gold_partecipants_df.loc[gold_partecipants_df.region == 'Not Found'])


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names


In [107]:
# Column Temporary aide in population search
gold_partecipants_df['population_help'] = gold_partecipants_df.NOC


# Replacement countries needed for population fill
# 'GDR' 'GER'
# 'ROT'
# 'VNM' 'VIE'
# 'EUN' 'RUS'
# 'YUG' 'SRB'
# 'IOA'
# 'URS' 'RUS'
# 'FRG' 'GER'
# 'YAR' 'YEM'
# 'SCG' 'SRB'
# 'TCH' 'CZE'
# 'YMD' 'YEM'
gold_partecipants_df['population_help'] = gold_partecipants_df['population_help'].replace(
    to_replace=['GDR', 'VNM', 'EUN', 'YUG', 'URS', 'FRG', 'YAR', 'SCG', 'TCH', 'YMD'],
    value=['GER', 'VIE', 'RUS','SRB', 'RUS',  'GER', 'YEM', 'SRB', 'CZE', 'YEM']
)

display(gold_partecipants_df)


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,population_help
0,AFG,1964,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,AFG
1,AFG,1968,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan,AFG
2,AFG,1972,8,8,0,0,8,0,8,0,Afghanistan,AFG,Afghanistan,AFG
3,AFG,1980,11,11,0,0,11,0,11,0,Afghanistan,AFG,Afghanistan,AFG
4,AFG,1988,5,5,0,0,5,0,5,0,Afghanistan,AFG,Afghanistan,AFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,ZIM,2000,26,16,0,0,19,0,19,0,Zimbabwe,ZWE,Zimbabwe,ZIM
2176,ZIM,2004,14,12,1,1,8,1,11,3,Zimbabwe,ZWE,Zimbabwe,ZIM
2177,ZIM,2008,16,13,0,1,11,3,15,4,Zimbabwe,ZWE,Zimbabwe,ZIM
2178,ZIM,2012,9,7,0,0,8,0,8,0,Zimbabwe,ZWE,Zimbabwe,ZIM


In [108]:
# Load population data
# From https://data.worldbank.org/indicator/SP.POP.TOTL
# and https://population.un.org/wpp/Download/Standard/Population/
population_df = pd.read_parquet(os.path.join(dataset_path, 'populations.parquet'))
population_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16409 entries, 2 to 16405
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  16409 non-null  object
 1   Country Code  16409 non-null  object
 2   Year          16409 non-null  int32 
 3   Population    16306 non-null  Int64 
dtypes: Int64(1), int32(1), object(2)
memory usage: 592.9+ KB


In [109]:
population_df = population_df.set_index(['Country Code', 'Year'])
display(population_df.head(5))

Country Name  Population
Country Code Year                         
AFG          1960  Afghanistan     8996967
             1961  Afghanistan     9169406
             1962  Afghanistan     9351442
             1963  Afghanistan     9543200
             1964  Afghanistan     9744772

In [110]:
# Fill population
test_pop = population_df.loc[(gold_partecipants_df.NOC, gold_partecipants_df.Year.astype(int)), 'Population'].reset_index()
gold_partecipants_df = gold_partecipants_df.merge(test_pop, left_on=['population_help', 'Year'], right_on=['Country Code', 'Year'], how='left')


In [111]:
# Cleaning up columns
gold_partecipants_df = gold_partecipants_df.drop(columns=['population_help', 'Country Code'])


In [112]:
# CHECKPOINT: my full db aggregated by country with no nulls
# Countries with no known population (like refugees olympic team) are discarded

# Countries with no population
no_population_df = gold_partecipants_df[gold_partecipants_df['Population'].isna()]
print(f'Number of rows with no population: {len(no_population_df)}')
display(no_population_df)

gold_partecipants_df = gold_partecipants_df.dropna(subset=['Population'])
gold_partecipants_df.to_parquet(os.path.join(dataset_path, 'gold_partecipants.parquet'))


Number of rows with no population: 8


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
604,ERI,2012,12,12,0,0,7,0,7,0,Eritrea,ERI,Eritrea,<NA>
605,ERI,2016,12,12,0,0,6,0,6,0,Eritrea,ERI,Eritrea,<NA>
921,IOA,1992,76,58,2,0,54,1,57,3,Individual Olympic Athletes,,None,<NA>
922,IOA,2000,4,4,0,0,4,0,4,0,Individual Olympic Athletes,,None,<NA>
923,IOA,2012,4,4,0,0,4,0,4,0,Individual Olympic Athletes,,None,<NA>
924,IOA,2016,9,9,1,1,6,0,8,2,Individual Olympic Athletes,,None,<NA>
1121,KUW,1992,37,32,0,0,18,0,18,0,Kuwait,KWT,Kuwait,<NA>
1667,ROT,2016,12,10,0,0,12,0,12,0,NaN,NaN,NaN,<NA>


In [113]:
# Entriees with no partecipants
print(f'Number of rows with no partecipants: {len(gold_partecipants_df[gold_partecipants_df.TotalPartecipants == 0])}')



Number of rows with no partecipants: 0


In [114]:
display(gold_partecipants_df.info())

# Metrics for countries those with one medal or more
gold_metrics_partecipants_df = gold_partecipants_df[gold_partecipants_df.TotalMedals > 0]
display(gold_metrics_partecipants_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2179
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   NOC                       2172 non-null   object
 1   Year                      2172 non-null   int64 
 2   events_partecipations_no  2172 non-null   int64 
 3   distinct_athletes_no      2172 non-null   int64 
 4   Bronze                    2172 non-null   int32 
 5   Gold                      2172 non-null   int32 
 6   NoMedal                   2172 non-null   int64 
 7   Silver                    2172 non-null   int32 
 8   TotalPartecipants         2172 non-null   int32 
 9   TotalMedals               2172 non-null   int32 
 10  region                    2160 non-null   object
 11  alpha_3                   2160 non-null   object
 12  iso_names                 2160 non-null   object
 13  Population                2172 non-null   Int64 
dtypes: Int64(1), int32(5), i

None

,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
7,AFG,2008,4,4,1,0,3,0,4,1,Afghanistan,AFG,Afghanistan,27722281
8,AFG,2012,6,6,1,0,5,0,6,1,Afghanistan,AFG,Afghanistan,31161378
15,AHO,1988,4,3,0,0,3,1,4,1,Curacao,CUW,Curaçao,149254
33,ALG,1984,33,33,2,0,15,0,17,2,Algeria,DZA,Algeria,21763578
35,ALG,1992,36,35,1,1,25,0,27,2,Algeria,DZA,Algeria,27028330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,ZAM,1984,18,16,1,0,17,0,18,1,Zambia,ZMB,Zambia,6701547
2163,ZAM,1996,8,8,0,0,7,1,8,1,Zambia,ZMB,Zambia,9339740
2170,ZIM,1980,52,42,0,1,29,0,30,1,Zimbabwe,ZWE,Zimbabwe,7408630
2176,ZIM,2004,14,12,1,1,8,1,11,3,Zimbabwe,ZWE,Zimbabwe,12019911


In [115]:
gold_metrics_partecipants_df = gold_metrics_partecipants_df.assign(
    PopulationPerMedal_thousands = np.divide(np.divide(gold_metrics_partecipants_df.Population, 1000), gold_metrics_partecipants_df.TotalMedals) )
gold_metrics_partecipants_df = gold_metrics_partecipants_df.assign(
    EventPartecipationPerMedal = np.divide(gold_metrics_partecipants_df.events_partecipations_no, gold_metrics_partecipants_df.TotalMedals) )
gold_metrics_partecipants_df = gold_metrics_partecipants_df.assign(
    AthletePerMedal = np.divide(gold_metrics_partecipants_df.distinct_athletes_no, gold_metrics_partecipants_df.TotalMedals) )
gold_metrics_partecipants_df = gold_metrics_partecipants_df.assign(
    AthletePerEventPartecipation = np.divide(gold_metrics_partecipants_df.distinct_athletes_no, gold_metrics_partecipants_df.events_partecipations_no) )

display(gold_metrics_partecipants_df.head(5))


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population,PopulationPerMedal_thousands,EventPartecipationPerMedal,AthletePerMedal,AthletePerEventPartecipation
7,AFG,2008,4,4,1,0,3,0,4,1,Afghanistan,AFG,Afghanistan,27722281,27722.281,4.0,4.0,1.000000
8,AFG,2012,6,6,1,0,5,0,6,1,Afghanistan,AFG,Afghanistan,31161378,31161.378,6.0,6.0,1.000000
15,AHO,1988,4,3,0,0,3,1,4,1,Curacao,CUW,Curaçao,149254,149.254,4.0,3.0,0.750000
33,ALG,1984,33,33,2,0,15,0,17,2,Algeria,DZA,Algeria,21763578,10881.789,16.5,16.5,1.000000
35,ALG,1992,36,35,1,1,25,0,27,2,Algeria,DZA,Algeria,27028330,13514.165,18.0,17.5,0.972222


In [116]:
gold_metrics_partecipants_df.drop(columns='Year').describe()

,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,Population,PopulationPerMedal_thousands,EventPartecipationPerMedal,AthletePerMedal,AthletePerEventPartecipation
count,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,8.610000e+02,8.610000e+02,861.000000,861.000000,861.000000
mean,162.288037,119.869919,4.441347,3.996516,66.403020,3.982578,78.823461,12.420441,5.759738e+07,1.661976e+04,25.607903,19.748040,0.785246
std,164.983912,119.404391,6.834275,8.565275,50.906791,7.022451,67.263930,21.663220,1.654930e+08,7.955359e+04,24.845350,18.492840,0.119667
min,3.000000,3.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,5.320000e+04,5.024067e+01,2.800000,2.384615,0.355263
25%,43.000000,36.000000,1.000000,0.000000,27.000000,0.000000,30.000000,2.000000,5.591572e+06,1.012278e+03,11.560000,8.684211,0.713287
50%,95.000000,74.000000,2.000000,1.000000,52.000000,1.000000,57.000000,4.000000,1.476009e+07,2.463843e+03,18.000000,14.142857,0.785571
75%,232.000000,163.000000,5.000000,4.000000,94.000000,4.000000,107.000000,13.000000,4.923058e+07,9.222692e+03,31.000000,24.000000,0.870370
max,839.000000,648.000000,46.000000,82.000000,234.000000,69.000000,322.000000,195.000000,1.378665e+09,1.129623e+06,281.000000,174.000000,1.000000


In [117]:
print('Gold:')
display(gold_partecipants_df.sort_values(by='Gold', ascending=False).head(5))
print('Silver:')
display(gold_partecipants_df.sort_values(by='Silver', ascending=False).head(5))
print('Bronze:')
display(gold_partecipants_df.sort_values(by='Bronze', ascending=False).head(5))
print('Total Medals:')
display(gold_partecipants_df.sort_values(by='TotalMedals', ascending=False).head(5))


Gold:


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
2082,USA,1984,693,522,30,82,135,61,308,173,United States,USA,United States,235825000
2063,URS,1980,660,489,46,80,104,69,299,195,Russia,SUN,"USSR, Union of Soviet Socialist Republics",139010000
2064,URS,1988,647,481,46,54,149,31,280,131,Russia,SUN,"USSR, Union of Soviet Socialist Republics",146857000
410,CHN,2008,730,599,28,51,204,21,304,100,China,CHN,China,1324655000
2061,URS,1972,531,371,22,50,127,27,226,99,Russia,SUN,"USSR, Union of Soviet Socialist Republics",131909000


Silver:


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
2063,URS,1980,660,489,46,80,104,69,299,195,Russia,SUN,"USSR, Union of Soviet Socialist Republics",139010000
2082,USA,1984,693,522,30,82,135,61,308,173,United States,USA,United States,235825000
2062,URS,1976,574,410,35,49,111,41,236,125,Russia,SUN,"USSR, Union of Soviet Socialist Republics",135147000
2087,USA,2004,726,533,26,36,211,39,312,101,United States,USA,United States,292805298
2088,USA,2008,763,588,35,36,212,39,322,110,United States,USA,United States,304093966


Bronze:


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
2064,URS,1988,647,481,46,54,149,31,280,131,Russia,SUN,"USSR, Union of Soviet Socialist Republics",146857000
2063,URS,1980,660,489,46,80,104,69,299,195,Russia,SUN,"USSR, Union of Soviet Socialist Republics",139010000
742,GDR,1980,495,346,42,47,104,37,230,126,Germany,DEU,Germany,78288576
2090,USA,2016,719,555,38,46,200,37,321,121,United States,USA,United States,323071755
2084,USA,1992,734,545,37,37,201,34,309,108,United States,USA,United States,256514000


Total Medals:


,NOC,Year,events_partecipations_no,distinct_athletes_no,Bronze,Gold,NoMedal,Silver,TotalPartecipants,TotalMedals,region,alpha_3,iso_names,Population
2063,URS,1980,660,489,46,80,104,69,299,195,Russia,SUN,"USSR, Union of Soviet Socialist Republics",139010000
2082,USA,1984,693,522,30,82,135,61,308,173,United States,USA,United States,235825000
2064,URS,1988,647,481,46,54,149,31,280,131,Russia,SUN,"USSR, Union of Soviet Socialist Republics",146857000
742,GDR,1980,495,346,42,47,104,37,230,126,Germany,DEU,Germany,78288576
2062,URS,1976,574,410,35,49,111,41,236,125,Russia,SUN,"USSR, Union of Soviet Socialist Republics",135147000


In [118]:
gold_metrics_partecipants_df.to_parquet(os.path.join('dataset','gold_countries_year_metrics.parquet'))